In [1]:
import json
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/machinelearning/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
class ModelBuilder(object):
    def __init__(self):
        with open('intents.json') as json_data:
            self.intents = json.load(json_data)
        self.words = []
        self.classes = []
        self.documents = []
        self.ignore_words = ['?']


    def parse_intents_doc(self):     
        # loop through each sentence in our intents patterns
        for intent in self.intents['intents']:
            for pattern in intent['patterns']:
                # tokenize each word in the sentence
                w = nltk.word_tokenize(pattern)
                # add to our words list
                self.words.extend(w)
                # add to documents in our corpus
                self.documents.append((w, intent['tag']))
                # add to our classes list
                if intent['tag'] not in self.classes:
                    self.classes.append(intent['tag'])

        # stem and lower each word and remove duplicates
        self.words = [stemmer.stem(w.lower()) for w in self.words if w not in self.ignore_words]
        self.words = sorted(list(set(self.words)))
        # remove duplicates
        self.classes = sorted(list(set(self.classes)))

    def build_training_data(self):
        # create our training data
        training = []
        output = []
        # create an empty array for our output
        output_empty = [0] * len(self.classes)

        # training set, bag of words for each sentence
        for doc in self.documents:
            # initialize our bag of words
            bag = []
            # list of tokenized words for the pattern
            pattern_words = doc[0]
            # stem each word
            pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
            # create our bag of words array
            for w in self.words:
                if w in pattern_words:
                    bag.append(1)
                else:
                    bag.append(0)

            # output is a '0' for each tag and '1' for current tag
            output_row = list(output_empty)
            output_row[self.classes.index(doc[1])] = 1

            training.append([bag, output_row])

        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training)

        # create train and test lists
        train_x = list(training[:,0])
        train_y = list(training[:,1])
        return train_x, train_y

    def train_neural_network(self, train_x, train_y):
        # reset underlying graph data
        tf.reset_default_graph()
        # Build neural network
        net = tflearn.input_data(shape=[None, len(train_x[0])])
        net = tflearn.fully_connected(net, 8)
        net = tflearn.fully_connected(net, 8)
        net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
        net = tflearn.regression(net)

        # Define model and setup tensorboard
        model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
        # Start training (apply gradient descent algorithm)
        model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
        model.save('model.tflearn')

        # save all of our data structures
        pickle.dump({
            'words': self.words,
            'classes': self.classes,
            'train_x': train_x,
            'train_y': train_y
          }, 
          open('training_data', 'wb')
        )

In [14]:
model_builder = ModelBuilder()
model_builder.parse_intents_doc()
train_x, train_y = model_builder.build_training_data()
model_builder.train_neural_network(train_x, train_y)

Training Step: 3999  | total loss: 0.27028 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.27028 - acc: 0.9856 -- iter: 24/25
Training Step: 4000  | total loss: 0.24576 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.24576 - acc: 0.9870 -- iter: 25/25
--
INFO:tensorflow:/Users/machinelearning/Documents/chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
data = pickle.load(open('training_data', 'rb'))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

# load our saved model
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.load('./model.tflearn')

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    print return_list
    return return_list

def response(sentence, userID='123', show_details=False):
    print sentence
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    print(random.choice(i['responses']))
                    return

            results.pop(0)

INFO:tensorflow:Restoring parameters from /Users/machinelearning/Documents/chatbot/model.tflearn


NotFoundError: Key Accuracy_1/Mean/moving_avg not found in checkpoint
	 [[Node: save_9/RestoreV2_3 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_9/Const_0_0, save_9/RestoreV2_3/tensor_names, save_9/RestoreV2_3/shape_and_slices)]]

Caused by op u'save_9/RestoreV2_3', defined at:
  File "/Users/machinelearning/anaconda/envs/gl-env/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/machinelearning/anaconda/envs/gl-env/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/machinelearning/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/machinelearning/anaconda/envs/gl-env/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/machinelearning/anaconda/envs/gl-env/lib/python2.7/site-packag

In [ ]:
response('how do I access CAT')